In [ ]:
def extract_features(labels, output_path, image):
    # Generate a colormap for labels
    label_cmap = colors.ListedColormap(np.random.rand(labels.max() + 1, 3))

    # Create a colored label image
    colored_label_image = label_cmap(labels)

    # Convert to uint8 format (0-255)
    colored_label_image = (colored_label_image * 255).astype(np.uint8)

    output_path = os.path.join(initial_path + "labeled", label, "labeled_" + file)
    if not os.path.exists(output_path):
        cv2.imwrite(output_path, cv2.cvtColor(colored_label_image, cv2.COLOR_RGBA2BGRA))

    info_table = pd.DataFrame(
        measure.regionprops_table(
            labels,
            intensity_image=image,
            properties=['label', 'area', 'mean_intensity', 'solidity', 'convex_area', 'equivalent_diameter', 'perimeter', 'extent', 'max_intensity', 'min_intensity', 'eccentricity'],
        )
    )

    occupancy = compute_occupancy(labels, image)
    info_table['occupancy'] = occupancy  # Add the occupancy to the statistics
    info_table = info_table.rename(columns={"label": "sample"})
    info_table['label'] = label
    info_table['image_name'] = file

    compute_feature_statistics(info_table, output_csv, properties_list)

In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
from skimage import exposure
from scipy import ndimage
from cellpose import models
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import filters
from skimage import morphology
from skimage import measure
from matplotlib import colors
from importlib import reload
import utility
reload(utility)
from utility import get_hard_disk_path, compute_big_cell_labels, compute_small_cell_labels

file = r"E:\data_for_seg\z_projection\split\s13.jpg"

image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
labels = compute_small_cell_labels(image)

info_table = pd.DataFrame(
    measure.regionprops_table(
        labels,
        intensity_image=image,
        properties=['label', 'area', 'mean_intensity', 'solidity', 'convex_area', 'equivalent_diameter', 'perimeter', 'extent', 'max_intensity', 'min_intensity', 'eccentricity'],
    )
)



In [10]:
import numpy as np
from scipy.spatial.distance import cdist
from skimage import measure

def calculate_closest_boundary_distances(labels):
    try:
        regions = measure.regionprops(labels)
        min_distances = []

        for i, region1 in enumerate(regions):
            min_distance = np.inf
            boundary1 = np.array(region1.coords)

            for j, region2 in enumerate(regions):
                if i != j:
                    boundary2 = np.array(region2.coords)

                    # Compute all pairwise distances between boundary points
                    distance_matrix = cdist(boundary1, boundary2)

                    # Find the minimum distance to this region
                    current_min_distance = np.min(distance_matrix)
                    if current_min_distance < min_distance:
                        min_distance = current_min_distance

            if min_distance != np.inf:
                min_distances.append(min_distance)

        average_min_distance = np.mean(min_distances)

        return average_min_distance

    except:
        return None

# Calculate closest boundary distances for each object
min_distances = calculate_closest_boundary_distances(labels)

# Compute average of these minimum distances
if min_distances:
    average_min_distance = np.mean(min_distances)
    print("Average of minimum distances to nearest boundary:", average_min_distance)
else:
    print("No distances calculated. There might be only one label.")


Average of minimum distances to nearest boundary: 95.25590343184498
